In [16]:
import numpy as np
from scipy.sparse.linalg import eigsh
from itertools import product
from collections import defaultdict
import time
import csv
import functions as func
import hamiltonianGenerator as hG

# Generating the possible combinations of states

In [ ]:
sites = 3
states = 2

In [11]:
matrix = np.zeros((8,8))
matrix[1,2] = 1
matrix

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [12]:
for p in range(0,8):
    for q in range(0,8):
        for r in range(0,8):
            matrix[r,q] = p + q*10 + r*100

matrix

array([[  7.,  17.,  27.,  37.,  47.,  57.,  67.,  77.],
       [107., 117., 127., 137., 147., 157., 167., 177.],
       [207., 217., 227., 237., 247., 257., 267., 277.],
       [307., 317., 327., 337., 347., 357., 367., 377.],
       [407., 417., 427., 437., 447., 457., 467., 477.],
       [507., 517., 527., 537., 547., 557., 567., 577.],
       [607., 617., 627., 637., 647., 657., 667., 677.],
       [707., 717., 727., 737., 747., 757., 767., 777.]])

In [15]:
def generate_matrix(site, state):
    states = ["".join(map(str, digits)) for digits in product(range(state), repeat=site)]
    matrix = []

    for row in states:
        row_entries = [f"{row}|{col}" for col in states]
        matrix.append(row_entries)

    return matrix


def print_matrix(matrix):
    for row in matrix:
        print(" ".join(row))

In [14]:
# Example usage
site = 3
state = 2
matrix = generate_matrix(site, state)
print_matrix(matrix)


000|000 000|001 000|010 000|011 000|100 000|101 000|110 000|111
001|000 001|001 001|010 001|011 001|100 001|101 001|110 001|111
010|000 010|001 010|010 010|011 010|100 010|101 010|110 010|111
011|000 011|001 011|010 011|011 011|100 011|101 011|110 011|111
100|000 100|001 100|010 100|011 100|100 100|101 100|110 100|111
101|000 101|001 101|010 101|011 101|100 101|101 101|110 101|111
110|000 110|001 110|010 110|011 110|100 110|101 110|110 110|111
111|000 111|001 111|010 111|011 111|100 111|101 111|110 111|111


# Converting Sheers matrix from m to p and saving it

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv(
    "/Users/gilfrim/Desktop/QuantumChemistryCoop/Main-CC-files/matrix_elements_K.csv",
    skiprows=1, header=0
)

# Clean column names
df.columns = [col.strip() for col in df.columns]

# Convert columns safely
df["m1"] = df["m1"].astype(int)
df["m2"] = df["m2"].astype(int)
df["<m1|K|m2>"] = df["<m1|K|m2>"].astype(float)

# Matrix size and init
size = max(df["m1"].max(), df["m2"].max()) + 1
K = np.zeros((size, size))

m = (size - 1) / 2

# Fill matrix
for _, row in df.iterrows():

    i = int(row["m1"])
    j = int(row["m2"])
    val = float(row["<m1|K|m2>"])
    K[i, j] = val
    K[j, i] = val


K_in_p = np.zeros((size, size))

for col in size:
    K_in_p[:,size] = K[:, m]
    
# Show matrix
print(pd.DataFrame(K))

      0     1    2    3    4    5    6    7    8     9     10
0   25.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
1    0.0  16.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
2    0.0   0.0  9.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
3    0.0   0.0  0.0  4.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
4    0.0   0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   0.0   0.0
5    0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0
6    0.0   0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0   0.0   0.0
7    0.0   0.0  0.0  0.0  0.0  0.0  0.0  4.0  0.0   0.0   0.0
8    0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  9.0   0.0   0.0
9    0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  16.0   0.0
10   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  25.0


array([[25.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., 16.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  9.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  4.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  9.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 16.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 25.]])